In [15]:
import pandas as pd
import statsmodels.api as sm
import seaborn as sns
#Airbnb Listing Data
df= pd.read_csv("airbnb.csv")
#Airbnb Calender Data
df1= pd.read_csv("Airbnb_listing.csv")

/Users/divyauppal/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (67) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
# Merging Calender and Listing Data
df3 = pd.merge(df1,df,how = "inner",left_on = "id", right_on = "listing_id",left_index = False, right_index = False)


In [ ]:
##Manhattan Data Cleaning and saving to be used in Excel/Tableau
df6 = df3[df3.neighbourhood_group_cleansed == "Manhattan"]
df7 = df6[((df6.accommodates == 2) & (df6.room_type == "Entire home/apt"))]
df7.drop(columns = ["listing_url","scrape_id","last_scraped","name","description","neighborhood_overview","picture_url","host_id"], inplace = True)
df7.drop(columns = ["host_url","host_name","host_location","host_about"], inplace = True)
df7.drop(columns = ["host_thumbnail_url","host_picture_url"], inplace = True)
df7.to_csv("Manhattan_Data.csv")

In [ ]:
# Saving data for midtown
print(df4[df4.duplicated()])
df4 = df3[df3.neighbourhood_cleansed == "Midtown"]
df4.to_csv("Midtown_Data.csv")

In [21]:
# Looking at 2 accomodation place in Midtown (Manhattan)
df5 = df4[((df4.accommodates == 2) & (df4.room_type == "Entire home/apt"))]
df5['len_amen'] = df5['amenities'].apply(lambda x: len(x.split(',')))
df5.to_csv("Midtown_Data_Home_2.csv")


In [ ]:
#Cleaning data for 2 accomodation place in Midtown
df_test = df5.copy(deep = True)
df_test.drop(columns = ['id','host_since','host_response_time','host_response_rate','host_acceptance_rate','price_x','host_total_listings_count','minimum_nights_y'], inplace = True)
df_test.dropna(subset=["bathrooms_text","price_y",'host_listings_count','bedrooms','beds','maximum_nights_x','minimum_nights_x','availability_30','availability_60','availability_90','availability_365','number_of_reviews','number_of_reviews_ltm'], axis=0, inplace=True)
df_test.dropna(subset=['number_of_reviews_l30d','review_scores_rating','review_scores_accuracy','review_scores_cleanliness','review_scores_checkin','review_scores_checkin','review_scores_location','review_scores_value','reviews_per_month','len_amen'], axis=0, inplace=True)
#df_test.dropna(inplace = True)
#df_test.dropna(inplace = True)

df_test['price_y'] = df_test['price_y'].apply(lambda x: x.replace('$',''))
df_test['bathrooms_text'] = df_test['bathrooms_text'].apply(lambda x: x.replace('bath',''))
df_test['price_y'] = df_test['price_y'].apply(lambda x: x.replace(',',''))


In [ ]:
#Checking factors which drive price of an Airbnb Listing
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV

X = df_test[['host_listings_count','bedrooms','beds','maximum_nights_x','minimum_nights_x','availability_365','number_of_reviews','review_scores_rating','review_scores_accuracy','review_scores_cleanliness','review_scores_checkin','review_scores_checkin','review_scores_location','review_scores_value','len_amen']]
Y = df_test["price_y"]
std = StandardScaler()
std.fit(X.values)
## apply the scaler to the train set
X_tr = std.transform(X.values)
## Apply the scaler to the test set

lasso = Lasso(alpha = 10)
lasso.fit(X_tr, Y)
test_r_squared = lasso.score(X_tr, Y)
print(test_r_squared)
print(list(zip(X.columns, lasso.coef_)))